In [36]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pp

In [37]:
df = pd.read_excel("data.xlsx", index_col="exp")
df.head()

,m1,m2,m3,s1__autocorrelation__lag_8,s1__autocorrelation__lag_9,s1__autocorrelation__lag_7,s1__autocorrelation__lag_6,s1__autocorrelation__lag_5,s1__autocorrelation__lag_4,s1__longest_strike_above_mean,...,s1__ar_coefficient__coeff_3__k_10,s1__approximate_entropy__m_2__r_0.1,s1__lempel_ziv_complexity__bins_3,s1__partial_autocorrelation__lag_4,"s1__fft_coefficient__attr_""abs""__coeff_7","s1__agg_autocorrelation__f_agg_""var""__maxlag_40",s1__spkt_welch_density__coeff_2,y1,y2,y3
exp,,,,,,,,,,,,,,,,,,,,,
E001,150.223716,1176.177278,1.142097,-0.305434,-0.519191,-0.074829,0.159896,0.385790,0.590387,14,...,0.183996,0.158567,0.204152,-0.360084,0.293617,0.499488,5.886812e-08,55.460434,1.065917,114.578620
E002,102.534268,1483.654982,1.104716,-0.243785,-0.454262,-0.021002,0.202836,0.416423,0.608972,14,...,0.184370,0.144742,0.203008,-0.344364,6.142373,0.477743,3.643621e-06,50.640306,1.285666,124.651484
E003,119.890549,1254.897451,2.162773,-0.329006,-0.543405,-0.095913,0.142612,0.373002,0.582151,14,...,0.184036,0.144268,0.208163,-0.364611,26.783283,0.506435,1.590028e-04,50.832405,1.154859,57.018054
E004,162.830799,1302.043195,1.308283,-0.065152,-0.266498,0.138913,0.337187,0.521401,0.683873,16,...,0.187213,0.137326,0.193662,-0.355441,33.227591,0.460547,7.926165e-04,62.476545,1.025161,132.221218
E005,165.720956,1154.482314,1.566830,-0.304881,-0.518177,-0.074836,0.159321,0.384728,0.589003,14,...,0.183978,0.128546,0.192440,-0.357588,11.439470,0.497940,1.462831e-04,57.634438,1.043776,92.160269


In [38]:
_X = df.iloc[:, :-3]
_Y = df.iloc[:, -3:]
print(_X.shape)
print(_Y.shape)

(100, 47)
(100, 3)


In [39]:
from sklearn.model_selection import train_test_split

_X_train, _X_test, _Y_train, _Y_test = train_test_split(
    _X, _Y, test_size=0.3, random_state=0
)
print(_X_train.shape)
print(_X_test.shape)
print(_Y_train.shape)
print(_Y_test.shape)

(70, 47)
(30, 47)
(70, 3)
(30, 3)


In [40]:
from sklearn.preprocessing import StandardScaler

scX = StandardScaler()
X_train = scX.fit_transform(_X_train)
X_test = scX.transform(_X_test)

scY = StandardScaler()
Y_train = scY.fit_transform(_Y_train)
Y_test = scY.transform(_Y_test)

In [41]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

reg = MultiOutputRegressor(estimator=SVR())

In [42]:
pp(reg.get_params())

{'estimator__C': 1.0,
 'estimator__cache_size': 200,
 'estimator__coef0': 0.0,
 'estimator__degree': 3,
 'estimator__epsilon': 0.1,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVR(),
 'n_jobs': None}


In [43]:
param_grid = [
    {
        "estimator__C": [1e-3, 0.1, 1, 10],
        "estimator__epsilon": [0.01, 0.1]
    }
]

In [44]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(estimator=reg, 
                  param_grid=param_grid,
                  cv=3,
                  scoring="neg_mean_squared_error",
                #   scoring='r2',
                  n_jobs=-1
                  )


In [45]:
gs.fit(X_train, Y_train)

,estimator,MultiOutputRe...timator=SVR())
,param_grid,"[{'estimator__C': [0.001, 0.1, ...], 'estimator__epsilon': [0.01, 0.1]}]"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,kernel,'rbf'


In [46]:
cv_results = gs.cv_results_

pd.DataFrame(data=cv_results)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__C,param_estimator__epsilon,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010082,0.000817,0.005194,0.000478,0.001,0.01,"{'estimator__C': 0.001, 'estimator__epsilon': ...",-0.839303,-1.171006,-1.064996,-1.025102,0.138324,8
1,0.009559,0.001877,0.003845,0.000236,0.001,0.10,"{'estimator__C': 0.001, 'estimator__epsilon': ...",-0.847772,-1.168824,-1.055743,-1.024113,0.132964,7
2,0.010224,0.001530,0.005714,0.002422,0.100,0.01,"{'estimator__C': 0.1, 'estimator__epsilon': 0.01}",-0.556798,-0.801398,-0.714098,-0.690765,0.101211,5
3,0.008695,0.001709,0.005071,0.001265,0.100,0.10,"{'estimator__C': 0.1, 'estimator__epsilon': 0.1}",-0.549840,-0.812857,-0.721066,-0.694588,0.108996,6
4,0.009397,0.000467,0.004352,0.000854,1.000,0.01,"{'estimator__C': 1, 'estimator__epsilon': 0.01}",-0.196048,-0.223566,-0.186451,-0.202021,0.015730,3
5,0.012737,0.002711,0.004023,0.002182,1.000,0.10,"{'estimator__C': 1, 'estimator__epsilon': 0.1}",-0.212154,-0.243097,-0.206639,-0.220630,0.016045,4
6,0.011250,0.001849,0.004187,0.001039,10.000,0.01,"{'estimator__C': 10, 'estimator__epsilon': 0.01}",-0.134100,-0.164669,-0.100730,-0.133166,0.026111,1
7,0.007712,0.000844,0.002711,0.000206,10.000,0.10,"{'estimator__C': 10, 'estimator__epsilon': 0.1}",-0.153165,-0.187289,-0.125531,-0.155328,0.025259,2


In [47]:
gs.best_estimator_.get_params()

{'estimator__C': 10,
 'estimator__cache_size': 200,
 'estimator__coef0': 0.0,
 'estimator__degree': 3,
 'estimator__epsilon': 0.01,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVR(C=10, epsilon=0.01),
 'n_jobs': None}

In [48]:
Y_train_pred = gs.predict(X_train)
Y_test_pred = gs.predict(X_test)

In [49]:
from datetime import datetime

dt = datetime.now().strftime("%Y-%m-%d_%H-%M")

In [50]:
SAVE_DATA = False

In [51]:
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score,
)


def eval_perf(y_true, y_pred):
    mse = mean_squared_error(y_true=y_true, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
    r2 = r2_score(y_true=y_true, y_pred=y_pred)
    return mse, mape, r2


def print_perf(data):
    for k, v in data.items():
        print(k, ":", v)


data_arr = []
for i in range(0, _Y.shape[1]):
    mse_train, mape_train, r2_train = eval_perf(
        y_true=Y_train[:, i], y_pred=Y_train_pred[:, i]
    )
    mse_test, mape_test, r2_test = eval_perf(
        y_true=Y_test[:, i], y_pred=Y_test_pred[:, i]
    )

    data = {
        "Y": f"Y-{i + 1}",
        "MSE Train": mse_train,
        "MSE Test": mse_test,
        "MAPE Train": mape_train,
        "MAPE Test": mape_test,
        "R2 Train": r2_train,
        "R2 Test": r2_test,
    }
    # print_perf(data)

    data_arr.append(data)

mse_train, mape_train, r2_train = eval_perf(y_true=Y_train, y_pred=Y_train_pred)
mse_test, mape_test, r2_test = eval_perf(y_true=Y_test, y_pred=Y_test_pred)
print_perf(data)

data = {
    "Y": "Y-All",
    "MSE Train": mse_train,
    "MSE Test": mse_test,
    "MAPE Train": mape_train,
    "MAPE Test": mape_test,
    "R2 Train": r2_train,
    "R2 Test": r2_test,
}
data_arr.append(data)
# print_perf(data)

df_eval = pd.DataFrame.from_dict(data_arr)
display(df_eval)

if SAVE_DATA:
    filename = f"eval_{dt}.xlsx"
    df_eval.to_excel(filename, index=False)

Y : Y-3
MSE Train : 9.658267211353761e-05
MSE Test : 0.16305111425738766
MAPE Train : 0.04827058653216231
MAPE Test : 0.5589407000647946
R2 Train : 0.9999034173278865
R2 Test : 0.8170864938530205


,Y,MSE Train,MSE Test,MAPE Train,MAPE Test,R2 Train,R2 Test
0,Y-1,0.000097,0.172726,0.029952,0.322854,0.999903,0.855570
1,Y-2,0.000093,0.108325,0.028382,0.362186,0.999907,0.865437
2,Y-3,0.000097,0.163051,0.048271,0.558941,0.999903,0.817086
3,Y-All,0.000095,0.148034,0.035535,0.414660,0.999905,0.846031
